In [64]:
import pandas as pd
import numpy as np

# Usage

In [25]:
usage = pd.read_csv('./20&21.csv', encoding='cp949')
usage.head()

,Unnamed: 0,자전거번호,반납일시,이용시간,이용거리
0,0,SPB-04061,2020-01-01,2.0,0.0
1,1,SPB-06686,2020-01-01,1.0,350.0
2,2,SPB-15937,2020-01-01,4.0,800.0
3,3,SPB-14805,2020-01-01,2.0,0.0
4,4,SPB-09038,2020-01-01,4.0,660.0


In [26]:
#필요없는 열 제거
usage.drop(['Unnamed: 0'], axis=1, inplace=True)

#홈페이지 기준 이용시간>240분인 자전거는 도난으로 취급
#이용시간>240인 데이터 삭제 (32331개 데이터 삭제 완료, 전체의 0.05%)
overtime = usage[usage['이용시간'] > 240].index
usage.drop(overtime, inplace=True)

#이용거리=NaN인 데이터는 데이터 자체가 밀려 수집된 것으로 확인
#이용거리=NaN인 값 삭제 (355개 데이터 삭제 완료)
usage = usage.dropna()

#이용시간, 이용거리 정수화
usage = usage.astype({'이용시간' : int, '이용거리' : int}, errors='raise')

#반납일시 datetime type
usage['반납일시'] = pd.to_datetime(usage['반납일시'])

#반납일시 열 이름을 등록일시로
usage.rename(columns={'반납일시': '등록일시'}, inplace=True)

In [27]:
usage.head()

,자전거번호,등록일시,이용시간,이용거리
0,SPB-04061,2020-01-01,2,0
1,SPB-06686,2020-01-01,1,350
2,SPB-15937,2020-01-01,4,800
3,SPB-14805,2020-01-01,2,0
4,SPB-09038,2020-01-01,4,660


In [28]:
usage.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54044163 entries, 0 to 54076493
Data columns (total 4 columns):
 #   Column  Dtype         
---  ------  -----         
 0   자전거번호   object        
 1   등록일시    datetime64[ns]
 2   이용시간    int32         
 3   이용거리    int32         
dtypes: datetime64[ns](1), int32(2), object(1)
memory usage: 1.6+ GB


In [29]:
usage.isna().sum()

자전거번호    0
등록일시     0
이용시간     0
이용거리     0
dtype: int64

# Breakdown

In [30]:
breakdown = pd.read_csv("C:/jupiter_workspace/ml_tp/bike_breakdown.csv", encoding='cp949')
breakdown.head()

,자전거번호,등록일시,고장구분
0,SPB-00108,2015-12-04,체인
1,SPB-00210,2015-12-07,체인
2,SPB-00035,2015-12-09,기타
3,SPB-01024,2015-12-13,기타
4,SPB-01015,2015-12-28,기타


In [31]:
#같은자전거에 같은 날에 여러건 등록된 데이터에 대해서는 첫행만 남기고 제거함
breakdown.drop_duplicates(['자전거번호','등록일시'], keep='first', inplace=True)

#등록일시 datetime type
breakdown['등록일시'] = pd.to_datetime(breakdown['등록일시'])

#고장구분 열 제거
breakdown.drop(['고장구분'], axis=1, inplace=True)


In [32]:
breakdown.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 297349 entries, 0 to 337871
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   자전거번호   297349 non-null  object        
 1   등록일시    297349 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 6.8+ MB


In [33]:
breakdown.isna().sum()

자전거번호    0
등록일시     0
dtype: int64

# 이제부터 시작

In [34]:
#쫄리니까 복사본 만들어야지
tuse = usage
tbreak = breakdown

In [36]:
#냅다 breakdown에 yes 만들기
tbreak['고장구분'] = 'Y'

#usage에 outer join으로 합치기
tmerge = tuse.merge(right=tbreak, how='outer', on=['자전거번호', '등록일시'], suffixes=('_',''))

#null값 있으면 다시 확인해보쟝
tmerge.isna().sum()


자전거번호           0
등록일시            0
이용시간       120169
이용거리       120169
고장구분     53080571
dtype: int64

이용시간 & 이용거리의 결측치 (120169 개) <br>
: 2019년 이전데이터에 대해 usage 데이터가 2020&2021년에 대해서만 수록되어 있어서 발생 -> 제거 <br>

고장구분의 결측치 (53080571 개) <br>
: 고장 안 난 날의 이용기록 -> 보존

In [49]:
#이용시간 & 이용거리의 결측치에 대해서는 삭제
tmerge.dropna(subset=['이용시간', '이용거리'], inplace=True)

#자전거번호, 등록일시, 고장구분이 같은 것 중 마지막 값만 남기고 제거
tmerge.drop_duplicates(['자전거번호', '등록일시', '고장구분'], keep='last', inplace=True)

#모든 행이 동일한 데이터가 있는지 확인
aaa = tmerge.duplicated(subset=['자전거번호', '등록일시', '이용시간', '이용거리', '고장구분'])
aa = aaa.sum()
print(aa) #0 >> 없음

#얘를 다시 usage와 left join
final_df = tuse.merge(right=tmerge, how='left', on=['자전거번호', '등록일시', '이용시간', '이용거리'], suffixes=('_',''))

final_df.head(30)

,자전거번호,등록일시,이용시간,이용거리,고장구분
0,SPB-04061,2020-01-01,2,0,NaN
1,SPB-06686,2020-01-01,1,350,NaN
2,SPB-15937,2020-01-01,4,800,NaN
3,SPB-14805,2020-01-01,2,0,NaN
4,SPB-09038,2020-01-01,4,660,NaN
5,SPB-18014,2020-01-01,4,970,NaN
6,SPB-12978,2020-01-01,4,910,NaN
7,SPB-17023,2020-01-01,2,450,NaN
8,SPB-14251,2020-01-01,3,910,NaN
9,SPB-18792,2020-01-01,5,1020,NaN


In [69]:
final_df.head(30)

,자전거번호,등록일시,이용시간,이용거리,고장구분
0,SPB-04061,2020-01-01,2,248.0,0
1,SPB-06686,2020-01-01,1,350.0,0
2,SPB-15937,2020-01-01,4,800.0,0
3,SPB-14805,2020-01-01,2,248.0,0
4,SPB-09038,2020-01-01,4,660.0,0
5,SPB-18014,2020-01-01,4,970.0,0
6,SPB-12978,2020-01-01,4,910.0,0
7,SPB-17023,2020-01-01,2,450.0,0
8,SPB-14251,2020-01-01,3,910.0,0
9,SPB-18792,2020-01-01,5,1020.0,0


In [68]:
#고장구분 NaN(고장안남)=0, Y(고장남)=1
#final_df['고장구분'] = final_df['고장구분'].fillna(0)
#final_df['고장구분'] = final_df['고장구분'].replace('Y', 1)

#이용거리=0이면 60분 미만인 값에 대해서는 같은 분의 평균, 
#60분 이상인 값은 평균 이용거리=3036.95*ln(이용시간)+-4210.63

#1~240분의 분 별 이동거리 평균값 리스트 생성
#mean_by_time = final_df.groupby(by='이용시간').mean()
#mean_by_time = mean_by_time.astype({'이용거리' : int}, errors='raise')
#240행까지 나옴. 1~240분 중 결측치 없음
#L_mean = mean_by_time['이용거리'].to_list()

for i in range(final_df.shape[0]) :
    if final_df.iloc[i, 3] == 0 :
        if final_df.iloc[i, 2] <= 60 : #60분 이하면
            usetime = final_df.iloc[i, 2]
            final_df.iloc[i, 3] = L_mean[usetime-1] #같은 이동시간에서의 이동거리 평균값
        else : #60분보다 크면
            usetime = final_df.iloc[i, 2]
            final_df.iloc[i, 3] = 3036.95*np.log(usetime)-4210.63 #회귀식

final_df.head(30)


KeyboardInterrupt: 

In [66]:
tfinal = final_df.head(30)

for i in range(tfinal.shape[0]) :
    if tfinal.iloc[i, 3] == 0 :
        if tfinal.iloc[i, 2] <= 60 :
            usetime = tfinal.iloc[i, 2]
            tfinal.iloc[i, 3] = L_mean[usetime-1]
        else :
            usetime = tfinal.iloc[i, 2]
            tfinal.iloc[i, 3] = 3036.95*np.log(usetime)-4210.63

tfinal

,자전거번호,등록일시,이용시간,이용거리,고장구분
0,SPB-04061,2020-01-01,2,248.0,0
1,SPB-06686,2020-01-01,1,350.0,0
2,SPB-15937,2020-01-01,4,800.0,0
3,SPB-14805,2020-01-01,2,248.0,0
4,SPB-09038,2020-01-01,4,660.0,0
5,SPB-18014,2020-01-01,4,970.0,0
6,SPB-12978,2020-01-01,4,910.0,0
7,SPB-17023,2020-01-01,2,450.0,0
8,SPB-14251,2020-01-01,3,910.0,0
9,SPB-18792,2020-01-01,5,1020.0,0


나중에 회귀선 그래프 사진 넣기~!!

밑에꺼 되는지 확인해야함~!!

In [ ]:
#이용거리 열 int 타입으로
final_df = final_df.astype({'이용거리' : int}, errors='raise')

#자전거이름, 대여일시 순으로 정렬
final_df.sort_values(by=['자전거이름', '대여일시'], ascending=True)

#누적 이용시간 & 누적 이동거리, 이용횟수 빈 열 생성
final_df.loc[:, '누적이용시간'] = np.nan
final_df.loc[:, '누적이용거리'] = np.nan
final_df.loc[:, '누적이용횟수'] = np.nan

#이용시간과 이동거리를 계속 합하다가 고장구분에 1을 만나면 기록
#자전거 넘버가 바뀌면 0으로 리셋, 다시시작

bikes = []
bikes = final_df['자전거번호'].unique()

for bike in bikes :
    cum_time = 0
    cum_dist = 0
    cum_use = 0

    bike_index = final_df[final_df['자전거번호'] == bike].index

    for k in bike_index :

        cum_time += final_df.iloc[k, 2] #누적이용시간
        cum_dist += final_df.iloc[k, 3] #누적이용거리
        cum_use += 1 #누적이용횟수

        if final_df.iloc[k, 4] == 1 : #고장났으면 이제까지 누적값 입력
            final_df.iloc[k, 5] == cum_time
            final_df.iloc[k, 6] == cum_dist
            final_df.iloc[k, 7] == cum_use



+++ 이용강도 이용거리 자전거나이 여름횟수

# 얘네는 대충 연습한거! 나 경계선 ---------------------------

In [51]:
tmerge[(tmerge['이용시간'] == 0)]

#aaa = tmerge.duplicated(subset=['자전거번호', '등록일시', '이용시간', '이용거리', '고장구분'])
#aa = aaa.sum()
#print(aa)

,자전거번호,등록일시,이용시간,이용거리,고장구분
